In [13]:
import requests
from bs4 import BeautifulSoup

In [19]:
#creating the object to hold to page source content
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168#.XpLOY8gzbcc")
#print(page)


#creating an object to store the page source in proper html format
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)

#storing page source belonging to a prticular id tag on the page
seven_day = soup.find(id="seven-day-forecast")
#print(seven_day)

#finding all instances of a particular class tag within the id and storing it to object
forecast_items = seven_day.find_all(class_="tombstone-container")
#print(forecast_items)

tonight = forecast_items[0]
print(tonight.prettify())


<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 89. Light and variable wind becoming south southeast around 5 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 89. Light and variable wind becoming south southeast around 5 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 89 °F
 </p>
</div>


In [20]:
#extracting the values of sub tags
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp temp-high").get_text()

print(period)
print(short_desc)
print(temp)

Today
Sunny
High: 89 °F


In [21]:
#extracting the title part of the img tag
img = tonight.find("img")
desc = img['title']

print(desc)

Today: Sunny, with a high near 89. Light and variable wind becoming south southeast around 5 mph. 


In [22]:
#extracting all values for the period tag from the entire object
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
print(periods)

['Today', 'Tonight', 'Friday', 'FridayNight', 'Saturday', 'SaturdayNight', 'Sunday', 'SundayNight', 'Monday']


In [28]:
period_tags

[<p class="period-name">Today<br/><br/></p>,
 <p class="period-name">Tonight<br/><br/></p>,
 <p class="period-name">Friday<br/><br/></p>,
 <p class="period-name">Friday<br/>Night</p>,
 <p class="period-name">Saturday<br/><br/></p>,
 <p class="period-name">Saturday<br/>Night</p>,
 <p class="period-name">Sunday<br/><br/></p>,
 <p class="period-name">Sunday<br/>Night</p>,
 <p class="period-name">Monday<br/><br/></p>]

In [29]:
#similarly extracting the other tags also from the entire object
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print()
print(temps)
print()
print(descs)

['Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny']

['High: 89 °F', 'Low: 64 °F', 'High: 92 °F', 'Low: 62 °F', 'High: 82 °F', 'Low: 58 °F', 'High: 75 °F', 'Low: 56 °F', 'High: 70 °F']

['Today: Sunny, with a high near 89. Light and variable wind becoming south southeast around 5 mph. ', 'Tonight: Mostly clear, with a low around 64. West wind 5 to 7 mph becoming calm  in the evening. ', 'Friday: Sunny, with a high near 92. Light and variable wind. ', 'Friday Night: Mostly clear, with a low around 62. West wind 5 to 9 mph becoming calm  in the evening. ', 'Saturday: Sunny, with a high near 82. Calm wind becoming south southeast around 5 mph in the afternoon. ', 'Saturday Night: Mostly clear, with a low around 58.', 'Sunday: Mostly sunny, with a high near 75.', 'Sunday Night: Partly cloudy, with a low around 56.', 'Monday: Mostly sunny, with a high near 70.']


In [30]:
#combining all tags and creating a dataframe out of it
import pandas as pd
weather = pd.DataFrame({
        "period": periods,
         "short_desc": short_descs,
         "temp": temps,
         "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 89 °F,"Today: Sunny, with a high near 89. Light and v..."
1,Tonight,Mostly Clear,Low: 64 °F,"Tonight: Mostly clear, with a low around 64. W..."
2,Friday,Sunny,High: 92 °F,"Friday: Sunny, with a high near 92. Light and ..."
3,FridayNight,Mostly Clear,Low: 62 °F,"Friday Night: Mostly clear, with a low around ..."
4,Saturday,Sunny,High: 82 °F,"Saturday: Sunny, with a high near 82. Calm win..."
5,SaturdayNight,Mostly Clear,Low: 58 °F,"Saturday Night: Mostly clear, with a low aroun..."
6,Sunday,Mostly Sunny,High: 75 °F,"Sunday: Mostly sunny, with a high near 75."
7,SundayNight,Partly Cloudy,Low: 56 °F,"Sunday Night: Partly cloudy, with a low around..."
8,Monday,Mostly Sunny,High: 70 °F,"Monday: Mostly sunny, with a high near 70."


In [31]:
#desc has the text that contains the value of period also
#trying to split the desc string such that the period part is removed
new = weather["desc"].str.split(": ",expand=True) 
print(new)

                0                                                  1
0           Today  Sunny, with a high near 89. Light and variable...
1         Tonight  Mostly clear, with a low around 64. West wind ...
2          Friday  Sunny, with a high near 92. Light and variable...
3    Friday Night  Mostly clear, with a low around 62. West wind ...
4        Saturday  Sunny, with a high near 82. Calm wind becoming...
5  Saturday Night                Mostly clear, with a low around 58.
6          Sunday                 Mostly sunny, with a high near 75.
7    Sunday Night               Partly cloudy, with a low around 56.
8          Monday                 Mostly sunny, with a high near 70.


In [32]:
#fetching the other part and replacing the desc variable 
weather["desc"]=new[1]

In [33]:
#updated dataframe
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 89 °F,"Sunny, with a high near 89. Light and variable..."
1,Tonight,Mostly Clear,Low: 64 °F,"Mostly clear, with a low around 64. West wind ..."
2,Friday,Sunny,High: 92 °F,"Sunny, with a high near 92. Light and variable..."
3,FridayNight,Mostly Clear,Low: 62 °F,"Mostly clear, with a low around 62. West wind ..."
4,Saturday,Sunny,High: 82 °F,"Sunny, with a high near 82. Calm wind becoming..."
5,SaturdayNight,Mostly Clear,Low: 58 °F,"Mostly clear, with a low around 58."
6,Sunday,Mostly Sunny,High: 75 °F,"Mostly sunny, with a high near 75."
7,SundayNight,Partly Cloudy,Low: 56 °F,"Partly cloudy, with a low around 56."
8,Monday,Mostly Sunny,High: 70 °F,"Mostly sunny, with a high near 70."


In [ ]:
weather.to_excel("weather_details.xlsx",header=True)